In [13]:
from langchain_community.document_loaders.generic import GenericLoader
from langchain_community.document_loaders.parsers import LanguageParser
from langchain_text_splitters import Language, RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.chains.question_answering import load_qa_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

import enum
import os
from git import Repo

In [2]:
repo_path = "./test_repo"

In [9]:
repo = Repo.clone_from("https://github.com/qcx/niamh", to_path=repo_path)

In [14]:
print([member for member in Language])

[<Language.CPP: 'cpp'>, <Language.GO: 'go'>, <Language.JAVA: 'java'>, <Language.KOTLIN: 'kotlin'>, <Language.JS: 'js'>, <Language.TS: 'ts'>, <Language.PHP: 'php'>, <Language.PROTO: 'proto'>, <Language.PYTHON: 'python'>, <Language.RST: 'rst'>, <Language.RUBY: 'ruby'>, <Language.RUST: 'rust'>, <Language.SCALA: 'scala'>, <Language.SWIFT: 'swift'>, <Language.MARKDOWN: 'markdown'>, <Language.LATEX: 'latex'>, <Language.HTML: 'html'>, <Language.SOL: 'sol'>, <Language.CSHARP: 'csharp'>, <Language.COBOL: 'cobol'>, <Language.C: 'c'>, <Language.LUA: 'lua'>, <Language.PERL: 'perl'>, <Language.HASKELL: 'haskell'>, <Language.ELIXIR: 'elixir'>, <Language.POWERSHELL: 'powershell'>]


In [17]:
loader = GenericLoader.from_filesystem(
    repo_path,
    glob="**/*",
    suffixes=[".ts"],
    exclude=[
        "**/moderation.ts",
        "**/statistics.ts",
        "**/storedFilters.ts",
        "**/discursive/**",
        "**/trainer/**"
    ],
    parser = LanguageParser(language=Language.TS, parser_threshold=500)
)

documents = loader.load()
len(documents)


21

[<Language.PYTHON: 1>, <Language.JAVA: 2>]
